# Code for practical part of PDC Project

Hard limits to respect :
- Codewords have to be in the space [-3/2, 3/2] x [-3/2, 3/2] -> **limits the energy per symbol**
- The average energy of the vector **X** should be <= 1 -> **also limits the energy per symbol** 
- The dimension of the encoded vector **X** should not exceed n <= 100 -> **limits the number of symbols we can send**

In [21]:
### IS THE ENERGY PER SYMBOL A HYPER-PARAMETER ???
import numpy as np
import scipy as sp

## Utility functions to help code the encoder/decoder

In [22]:
# Computes the value of the Q-Function
def q_function(x):
    return (1/2) * sp.erfc(x/np.sqrt(2))

# Gets every possible codeword lengths for which the length divides the total length of the original bitstring
def compute_codeword_lengths(bit_str):
    n = len(bit_str)
    return [i for i in range(1, n+1) if n % i == 0]

# Gets the whole alphabet from a codeword length
def get_alphabet_from_codeword_length(length):
    return [f'{i:0{length}b}' for i in range(0, 2**length)]

# Performs a conversion from ASCII string to binary symbols
def ascii_str_to_binary(ascii_str):
    return str(''.join(format(ord(i), '08b') for i in ascii_str))

def split_bit_str(bit_str, chunk_size):
    return [bit_str[i:i+chunk_size] for i in range(0, len(bit_str), chunk_size)]


# # Gets the encoded alphabet
# def encode_alphabet(energy_per_symbol, alphabet, encoder):
#     return [encoder(energy_per_symbol, i, alphabet) for i in alphabet]


## Functions for the types of encodings

### m-PSK

In [23]:
# Computes the probability of error for symbol i with PSK encoding (P_e(i) == P_e here !)
def error_m_psk(i, m, energy_per_symbol, noise_var = 10**(-2.65)):
    inner_part = lambda theta: np.exp(-(np.sin(np.pi/m)**2)**2 / np.sin(theta) * (energy_per_symbol/(2*(noise_var**2))))
    return (1/np.pi) * sp.quad(inner_part, 0, np.pi - (np.pi / m))

# Performs the m-PSK encoding
def m_psk_encoder(energy_per_symbol, codeword, constellation, alphabet, d=0):
    k = alphabet.index(codeword)
    m = len(alphabet)
    return np.sqrt(energy_per_symbol) * np.exp(2j * np.pi * (k/m))

### QAM

In [45]:
# Computes the probability of error for symbol i with QAM encoding (P_e(i) == P_e here !)
def error_qam(m, d, noise_var = 10**(-2.65)):
    func = q_function(d/(2*noise_var))
    return 2*func - func**2

# Computes the PAM constellation for a given number of points and a distance for each of these points.
def pam(n, d):
    right_side = np.array([d*i + (d/2) for i in range(0, int(n/2))])
    left_side = -right_side
    return np.append(np.sort(left_side), right_side)


# Computes the whole m-QAM constellation, to be used to map each codeword to a point generated here
# WARNING: m MUST BE A POWER OF 2 HERE !!!
def m_qam(m, d, max_energy = (3/2)):
    if d > max_energy:
        raise OverflowError("ERROR: the distance you have set is too big for the project.")

    axis_len = int(np.sqrt(m))
    if axis_len % 2 != 0:
        axis_len = axis_len + 1

    diff_with_nearest_square = m - axis_len**2

    # if the difference is positive => we need to add some more points
    if diff_with_nearest_square > 0:
        real_axis_pam = pam(diff_with_nearest_square, d)
    else:
        real_axis_pam = pam(axis_len, d)
    if real_axis_pam.max() > max_energy:
        raise OverflowError("ERROR: You have some points that are outside the constrained square. Try to lower the distance or reduce the size of the constellation.")
    
    imaginary_axis_pam = pam(axis_len, d)[::-1]
    points_pairs = np.array([[complex(re, im) for re in real_axis_pam] for im in imaginary_axis_pam])

    # if the difference is negative => we need to remove some points from the constellation
    if diff_with_nearest_square < 0:
        nb_elts_to_delete_per_half_row = np.sqrt(-diff_with_nearest_square)/2
        nb_rows_to_modify = -diff_with_nearest_square / (2*nb_elts_to_delete_per_half_row)

        # First we set them to 0
        for j in range(0, int(nb_rows_to_modify/2)):
            for i in range(0, int(nb_elts_to_delete_per_half_row)):
                points_pairs[j, i] = 0
                points_pairs[j, len(points_pairs)-1-i] = 0
                points_pairs[len(points_pairs)-1-j, i] = 0
                points_pairs[len(points_pairs)-1-j, len(points_pairs)-1-i] = 0

    # We flatten the array to make it easier to map the codewords
    points_pairs = points_pairs.flatten()

    # Then, we remove all the points set to 0 (impossible to have if the dimension is a full square, so no worries)
    points_pairs = np.delete(points_pairs, np.argwhere(points_pairs == 0.0+0.0j))

    return points_pairs


# Performs the QAM encoding (in a linear fashion for the moment => might be re-worked)
def m_qam_encoder(codeword, constellation, alphabet, d, energy_per_symbol=0):
    try:
        k = alphabet.index(codeword)
        m = len(alphabet)
        return constellation[k]
    except OverflowError as ovferr:
        raise

## Encoder part

In [33]:
def encode_string(raw_str, codeword_size, theta_estimator_batch_size, encoder, d = 1, energy_per_symbol = 1):
    if theta_estimator_batch_size >= 100:
        raise OverflowError("ERROR: The batch of dummy samples cannot be equal or exceed 100 symbols.") 
    bit_str = ascii_str_to_binary(raw_str)
    splitted_bit_str = split_bit_str(bit_str, codeword_size)
    splitted_bit_str_size = len(splitted_bit_str)
    if splitted_bit_str_size > 100:
        raise OverflowError("ERROR: The string without the dummy symbols cannot exceed 100 symbols.") 

    if splitted_bit_str_size + theta_estimator_batch_size > 100:
        theta_estimator_batch_size = 100 - splitted_bit_str_size

    alphabet = get_alphabet_from_codeword_length(codeword_size)
    theta_estimator_batch = np.full((theta_estimator_batch_size, 1), alphabet[0])
    full_str = np.append(theta_estimator_batch, splitted_bit_str)

    if encoder == m_qam_encoder:
        constellation = m_qam(2**codeword_size, d)
    else: # FOR PSK COMPUTING THE CONSTELLATION IS USELESS
        constellation = []

    return np.array([encoder(energy_per_symbol=energy_per_symbol, codeword=codeword, constellation=constellation, alphabet=alphabet, d=d) for codeword in full_str])

## Channel part (given)

In [26]:
def channel(sent_signal):
    s = np.mean(sent_signal**2)
    if s <= 1:
        s = 1
    noise_power = (10**(-2.65))*s
    shift = np.exp(-2j*np.pi*np.random.rand())
    sent_signal = sent_signal*shift
    noise_std = np.sqrt(noise_power/2)
    rcv_signal = sent_signal + noise_std*np.random.randn(len(sent_signal)) + 1j*noise_std*np.random.randn(len(sent_signal))
    return rcv_signal

## Decoder part

In [27]:
def decode_str(noisy_str, decoder, n_dummy_symbols):
    dummy_symbols = noisy_str[0:n_dummy_symbols]
    phase = np.mean(dummy_symbols) # TODO: CHECK HOW TO ISOLATE THE ANGLE FROM THE EXPONENTIAL IN THE EQUATION WITH THE MEAN
    return

## Putting everything together

In [49]:
str_to_process = "hellohowareyoutodayimtryingtodecodethestringliketheonegivenattheexambutitshard"

print(f"Possible codeword sizes for the string {str_to_process} of length {len(str_to_process)}: ", compute_codeword_lengths(ascii_str_to_binary(str_to_process)))

try:
    encoded_str = encode_string(str_to_process, codeword_size = 8, energy_per_symbol = 1, theta_estimator_batch_size = 20, d = 0.1, encoder = m_qam_encoder)
except OverflowError as ovferr :
    print(ovferr)
    raise
else:
    print(encoded_str)
    print()
    noisy_result = channel(encoded_str)
    print(noisy_result)


Possible codeword sizes for the string hellohowareyoutodayimtryingtodecodethestringliketheonegivenattheexambutitshard of length 78:  [1, 2, 3, 4, 6, 8, 12, 13, 16, 24, 26, 39, 48, 52, 78, 104, 156, 208, 312, 624]
[-0.75+0.75j -0.75+0.75j -0.75+0.75j -0.75+0.75j -0.75+0.75j -0.75+0.75j
 -0.75+0.75j -0.75+0.75j -0.75+0.75j -0.75+0.75j -0.75+0.75j -0.75+0.75j
 -0.75+0.75j -0.75+0.75j -0.75+0.75j -0.75+0.75j -0.75+0.75j -0.75+0.75j
 -0.75+0.75j -0.75+0.75j  0.05+0.15j -0.25+0.15j  0.45+0.15j  0.45+0.15j
  0.75+0.15j  0.05+0.15j  0.75+0.15j -0.05+0.05j -0.65+0.15j -0.55+0.05j
 -0.25+0.15j  0.15+0.05j  0.75+0.15j -0.25+0.05j -0.35+0.05j  0.75+0.15j
 -0.35+0.15j -0.65+0.15j  0.15+0.05j  0.15+0.15j  0.55+0.15j -0.35+0.05j
 -0.55+0.05j  0.15+0.05j  0.15+0.15j  0.65+0.15j -0.05+0.15j -0.35+0.05j
  0.75+0.15j -0.35+0.15j -0.25+0.15j -0.45+0.15j  0.75+0.15j -0.35+0.15j
 -0.25+0.15j -0.35+0.05j  0.05+0.15j -0.25+0.15j -0.45+0.05j -0.35+0.05j
 -0.55+0.05j  0.15+0.15j  0.65+0.15j -0.05+0.15j  0.45+0.